## 1. Import the Pandas Library

In [17]:
import numpy as np
import pandas as pd

## 2. Exploratory Data Analysis

In [18]:
# Read the data into a data frame
data = pd.read_csv('kc_house_data.csv')

In [19]:
# Check the number of data points in the data set
print(len(data))
# Check the number of features in the data set
print(len(data.columns))
# Check the data types available in this dataset
print(data.dtypes.unique())

21613
21
[dtype('int64') dtype('O') dtype('float64')]


We have one both numerical and categorical columns in this dataset.

## 3. Check Categorical Features
We can get the categorical column list using the below code

In [20]:
data.select_dtypes(include=['O']).columns.tolist()

['date']


We only have one categorical columns which is the date column that we will ignore. If you are interested to fine tune this model further, then you can preprocess this column. 


In [21]:
#view sample data
data.sample(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
18881,6413100242,20140826T000000,400000.0,3,1.75,1730,9211,1.0,0,0,...,8,1730,0,1961,0,98125,47.7149,-122.322,1440,9211
19232,3751604653,20140826T000000,205000.0,3,1.00,1370,10708,1.0,0,0,...,7,1370,0,1969,0,98001,47.2769,-122.264,1770,14482
19551,5028602020,20150305T000000,255000.0,3,2.25,1850,7151,2.0,0,0,...,7,1850,0,1989,0,98023,47.2843,-122.352,1710,6827
19210,1901600090,20150426T000000,390000.0,5,1.75,1940,6654,1.5,0,0,...,7,1940,0,1953,0,98166,47.4663,-122.359,2300,9500
6158,1223039173,20150429T000000,450000.0,4,1.75,2190,11625,1.0,0,0,...,8,2020,170,1956,0,98146,47.4979,-122.363,1920,8855


we don not need the id column also. So drop the column while fitting the data in ML model

## 4. Missing Value Columns List

In [22]:
#get the dimension of dataset
data.shape

(21613, 21)

In [23]:
#get record count for each variable
data.count()

id               21613
date             21613
price            21613
bedrooms         21613
bathrooms        21613
sqft_living      21613
sqft_lot         21613
floors           21613
waterfront       21613
view             21613
condition        21613
grade            21613
sqft_above       21613
sqft_basement    21613
yr_built         21613
yr_renovated     21613
zipcode          21613
lat              21613
long             21613
sqft_living15    21613
sqft_lot15       21613
dtype: int64

This dataset contains no missing value. So we can proceed further to make the machine learning model using the lightgbm

## 5. Split the Dataset

In [24]:
#separte x and y variable
X_data = data[['sqft_living','grade', 'sqft_above', 'sqft_living15','bathrooms','view',
                 'sqft_basement','lat','waterfront','yr_built','bedrooms']]
X=X_data.values
y = data.price.values

In [25]:
# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [26]:
#get train and test dataset size
X_train.shape


(17290, 11)

In [27]:
X_test.shape

(4323, 11)

17k records for training and 4k records for testing model efficiency.

## 6. CatBoost Regression Model

In [28]:
# import catboost library
from catboost import CatBoostRegressor

To know more about CatBoost 
https://tech.yandex.com/catboost/doc/dg/concepts/python-installation-docpage/
https://tech.yandex.com/catboost/doc/dg/concepts/python-quickstart-docpage/#regression

In [29]:
modelc=CatBoostRegressor(iterations=200,
                            learning_rate=0.6,
                            depth=8,
                            loss_function='RMSE',
                            eval_metric='RMSE',
                            random_seed=99,
                            od_type='Iter',
                            od_wait=50)
modelc.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True, verbose=False)

In [30]:
#predict the values
y_pred = modelc.predict(X_test)

## 7. Computer Error Metrics

In [31]:

# Evaluating the Algorithm
from sklearn.metrics import explained_variance_score
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('Explained Variance',explained_variance_score(y_test,y_pred))
print('R Square :', metrics.r2_score(y_test, y_pred))  


Mean Absolute Error: 87852.5034367
Mean Squared Error: 21712306616.1
Root Mean Squared Error: 147350.964083
Explained Variance 0.817427881063
R Square : 0.817427454089
